In [ ]:
from gurobipy import *
import numpy as np
import math as mt


In [ ]:
# Données
service_count=2
project_count=4
n = 2
p=4
budget_limit = 100
constraint_count=n*n +1
variable_count= n*n + n  + p
couts_projet = np.array([40, 50, 60, 50])

In [ ]:
# Matrice d'utilte
matrice_utilite = np.array([[19, 6, 17, 2], [2, 11, 4, 18]])
print(matrice_utilite)

In [ ]:
# vecteur w
w = [2, 1]

# creer le vecteur w'
w_prime = [(w[k] - w[k+1]) for k in range(len(w)-1)]
w_prime.append(w[-1])
print(w_prime)

In [ ]:
# matrice utilite
u = matrice_utilite

In [ ]:
## les variables rk
coef_rk = []
for k in range(n):
        for i in range(n):
                coef_rk.append([])
                for j in range(n):
                        coef_rk[-1].append(0)
                coef_rk[-1][k] = 1
coef_rk = np.array(coef_rk)
print(coef_rk)

In [ ]:
## les variables bik
coef_bik = -1 * np.identity(n**2)
print(coef_bik)

In [ ]:
## les variables x
coef_xi = []
for k in range(n**2):
        coef_xi.append([])
        for i in range(p):
                coef_xi[-1].append(-u[k%n][i])
print(coef_xi)
coef_xi = np.array(coef_xi)

In [ ]:
# contrainte sur le budget
budget = [0 for i in range(n*(n+1))]
budget += [couts_projet[i] for i in range(p)]
budget = np.array(budget)
print(budget)

In [ ]:
# concatenation des matrices rk, bik, xk 
result_concat = np.concatenate(
        (coef_rk, np.concatenate((coef_bik, coef_xi),axis=1)),
        axis=1)

# ajout de la contrainte sur les budgets
constrainte = np.vstack([result_concat, budget])
print(constrainte)

In [ ]:
## second memebre
second_membre = [0 for _ in range(n**2)]
second_membre.append(budget_limit)
second_membre = np.array(second_membre)
print (second_membre)

In [ ]:
# Coefficient de la fonction objectif
## vecteur w'
w_prime = [(w[k] - w[k+1]) for k in range(len(w)-1)]
w_prime.append(w[-1])

## coefficient fonction objectif
c = [(i+1) * w_prime[i] for i in range(n)] # coef des r_k
# coef des b_ik
for i in range(n) :
    for j in range(n) :
        c.append(-1*w_prime[i])

c += [0 for i in range(p)] # ajout des coef de x
c = np.array(c)
print(c)

In [ ]:
# Creer les models de la fonction objectif
m = Model("exercice1.3")

In [ ]:
# declaration des variables de decisions

x = []
for i in range(n):
    x.append(m.addVar(vtype=GRB.CONTINUOUS, lb=0, name="r%d" % (i+1)))
for i in range (n):
    for j in range(n):
        x.append(m.addVar(vtype=GRB.CONTINUOUS, lb=0, name="b%d%d" % ((j+1,(i+1)))))
for j in range(p):
    x.append(m.addVar(vtype=GRB.BINARY, lb=0, name="x%d" % (j+1) ))

In [ ]:
# maj du modele pour integrer les nouvelles variables
m.update()
print(x)

In [ ]:
# definition de l'objectif
obj = LinExpr();
obj =0
for j in range(variable_count):
    obj += c[j] * x[j]

m.setObjective(obj,GRB.MAXIMIZE)

In [ ]:
# Definition des contraintes
# Definition des contraintes
lignes = range(constraint_count)
colonnes = range(variable_count)
print (lignes, colonnes)
for i in lignes:
    m.addConstr(quicksum(constrainte[i][j]*x[j] for j in colonnes) <= second_membre[i], "Contrainte%d" % i)

In [ ]:
# Resolution
m.optimize()


print("")                
print('Solution optimale:')
mvars = m.getVars()
names = m.getAttr('VarName', mvars)
values = m.getAttr('X', mvars)
result = dict(zip(names, values))
for key in result:
    print(key,'=',result[key])
print("")
print('Valeur de la fonction objectif :', m.objVal)